In [3]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_classic.chains import LLMChain, SequentialChain
from langchain_core.prompts import PromptTemplate

In [4]:

llm = AzureChatOpenAI(
    azure_deployment=os.getenv("DEPLOYMENT_NAME"),
    azure_endpoint=os.getenv("OPENAI_API_ENDPOINT"),
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    temperature=0,
    max_retries=5,
)

In [5]:
chain1_prompt = PromptTemplate(
    input_variables = ["raw_material"],
    template="""
    You are a cosmetic scientist. Extract key properties of the ingredient '{raw_material}'.

    Return as structured text:
    - Origin (natural/synthetic/source)
    - Function (what it does in formulations)
    - Skin/Hair Benefit (main effects)

    Format clearly.
    """ 
)

chain1 = LLMChain(
    llm = llm,
    prompt = chain1_prompt,
    output_key = "ingredient_properties"
)

C:\Users\NibeditaKar\AppData\Local\Temp\ipykernel_5700\2206476762.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain1 = LLMChain(


In [6]:
chain2_prompt = PromptTemplate(
    input_variables = ["raw_materials", "ingredient_properties"],
    template = """
    You are a cosmetic formulator. Based on the ingredient '{raw_material}' and its properties:

    {ingredient_properties}

    1. Recommend which products this ingredient is best suited for.
    2. Suggest formulation ideas or concentration ranges.
    3. Keep it clear, structured, and practical.
    """
)

chain2 = LLMChain(
    llm = llm,
    prompt = chain2_prompt,
    output_key = "formulation_recommendations"
)

In [7]:
ingredient_pipeline = SequentialChain(
    chains = [chain1, chain2],
    input_variables = ["raw_material"],
    output_variables = ["ingredient_properties", "formulation_recommendations"],
    verbose = True
)

In [10]:
raw_material_input = "Niacinamide"

outputs = ingredient_pipeline.invoke({"raw_material": raw_material_input})

print("\n--- Chain 1 Output (Properties) ---\n")
print(outputs["ingredient_properties"])

print("\n--- Chain 2 Output (Formulation) ---\n")
print(outputs["formulation_recommendations"])

ingredient_pipeline.get_graph().print_ascii()



> Entering new SequentialChain chain...

> Finished chain.

--- Chain 1 Output (Properties) ---

**Niacinamide (Vitamin B3)**  

- **Origin**:  
  Synthetic (can be derived from nicotinic acid or synthesized in laboratories).  

- **Function**:  
  - Acts as a humectant and stabilizer in formulations.  
  - Improves the texture and stability of cosmetic products.  
  - Compatible with a wide pH range, making it versatile in formulations.  

- **Skin/Hair Benefit**:  
  - **Skin**:  
    - Reduces the appearance of enlarged pores.  
    - Improves uneven skin tone and dullness.  
    - Strengthens the skin barrier by boosting ceramide production.  
    - Minimizes fine lines and wrinkles.  
    - Reduces redness and inflammation (anti-inflammatory properties).  
    - Regulates sebum production, making it beneficial for oily skin.  
    - Helps fade hyperpigmentation and dark spots.  
  - **Hair**:  
    - Improves scalp health by reducing irritation and inflammation.  
    - Strength